In [2]:
import numpy as np 
import pandas as pd
import geopandas as gpd
import shapely
from shapely.wkt import loads
import shapely.vectorized as sv 
from scipy.spatial.distance import cdist 

In [3]:
# read bolinas nodes
nodes_df = pd.read_csv('../network_inputs/osm_nodes.csv')
nodes_gdf = gpd.GeoDataFrame(nodes_df, geometry=[shapely.geometry.Point(xy) for xy in zip(nodes_df.lon, nodes_df.lat)], crs='epsg:4326')
display(nodes_gdf.head(2))
# read visitor nodes
visitor_nodes_df = pd.read_csv('../network_inputs/osm_nodes_visitor_parking.csv')
visitor_nodes_gdf = gpd.GeoDataFrame(visitor_nodes_df, geometry=[shapely.geometry.Point(xy) for xy in zip(visitor_nodes_df.lon, visitor_nodes_df.lat)], crs='epsg:4326')
display(visitor_nodes_gdf.head(2))

,node_id_igraph,node_osmid,lon,lat,signal,geometry
0,0,110418187,-122.649946,37.902311,NaN,POINT (-122.64995 37.90231)
1,1,110392834,-122.711450,37.901207,NaN,POINT (-122.71145 37.90121)


,node_id_igraph,node_osmid,lon,lat,signal,geometry
0,7,110421520,-122.686998,37.906398,NaN,POINT (-122.68700 37.90640)
1,11,110421540,-122.687898,37.904398,NaN,POINT (-122.68790 37.90440)


In [4]:
### Get the OSMid of the closest node to each parcel
bolinas_parcels = pd.read_csv('parcels/parcels_bolinas/parcels_bolinas.csv')
household_parcels = bolinas_parcels[bolinas_parcels['UseCd']==11].reset_index() ### only UseCd 11 has household size>0 (=1)
household_parcels = gpd.GeoDataFrame(household_parcels, crs='epsg:4326',
                                     geometry=household_parcels['WKT'].apply(loads))

household_parcels['centroid'] = household_parcels.apply(lambda x: x['geometry'].centroid, axis=1)
household_parcels = household_parcels.set_geometry('centroid')
household_parcels['c_x'] = household_parcels['centroid'].x
household_parcels['c_y'] = household_parcels['centroid'].y
nodes_xy = np.array([nodes_gdf['geometry'].x.values, nodes_gdf['geometry'].y.values]).transpose()
nodes_osmid = nodes_gdf['node_osmid'].values

def get_closest_node(parcel_x, parcel_y):
    return nodes_osmid[cdist([(parcel_x, parcel_y)], nodes_xy).argmin()]

household_parcels['closest_node'] = household_parcels.apply(lambda x: get_closest_node(x['c_x'], x['c_y']), axis=1)
display(household_parcels.head(2))
household_parcels.shape
# parcels_evac.to_csv(absolute_path+'/outputs/parcels_evac.csv', index=False)

,index,WKT,WKT.1,Acres,UseCd,UseCdDesc,Parcel,AssessorSi,SHAPE_STAr,SHAPE_STLe,Prop_ID,geometry,centroid,c_x,c_y,closest_node
0,7,"POLYGON ((-122.705885636748 37.9437779330562,-...",MULTIPOLYGON (((-122.705885636748 37.943777933...,0.504,11,Single-Resid. - Improved,18810004,5855 STATE ROUTE 1,21985.887695,660.043465,188-100-04,"POLYGON ((-122.70589 37.94378, -122.70621 37.9...",POINT (-122.70613 37.94406),-122.706126,37.944060,110396428
1,8,"POLYGON ((-122.705569630863 37.9429819296033,-...",MULTIPOLYGON (((-122.705569630863 37.942981929...,2.124,11,Single-Resid. - Improved,18810005,5775 STATE ROUTE 1,92541.282227,1226.858154,188-100-05,"POLYGON ((-122.70557 37.94298, -122.70676 37.9...",POINT (-122.70628 37.94326),-122.706284,37.943258,110396428


(595, 16)

In [12]:
random_seed = 0
np.random.seed(random_seed)
vphh = 0
visitor_cnts = 0 ### 300 vehicles
commscen_id = 1

### local residents origin
local_residents_od_df = household_parcels[['Parcel', 'closest_node']].copy().sample(frac=1).reset_index(drop=True)
local_residents_od_df['origin_osmid'] = local_residents_od_df['closest_node']
local_residents_od_df = local_residents_od_df.drop('closest_node', 1)
local_residents_od_df['dept_time'] = 0
local_residents_od_df['type'] = 'local'

### visitor origin
visitor_vehicle_origin = np.random.choice(visitor_nodes_gdf['node_osmid'], size=visitor_cnts)
visitor_vehicle_od_df = pd.DataFrame({'origin_osmid': visitor_vehicle_origin})
visitor_vehicle_od_df = visitor_vehicle_od_df.sample(frac=1).reset_index(drop=True)
visitor_vehicle_od_df['dept_time'] = 0
visitor_vehicle_od_df['type'] = 'visitor'
visitor_vehicle_od_df['Parcel'] = None
od_df = pd.concat([local_residents_od_df]*vphh+[visitor_vehicle_od_df])

# if communication scenario 1, can go north or south
if commscen_id == 0:
    od_df['destin_osmid'] = np.random.choice(['110360959', '110397253'], size=od_df.shape[0])
else:
    od_df['destin_osmid'] = '110397253'
od_df.to_csv('od_csv/resident_visitor_od_rs{}_vphh{}_visitor{}_comm{}.csv'.format(random_seed, vphh, visitor_cnts, commscen_id), index=False)